# PDF Read In

# LDA Analysis

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 24 16:04:44 2019

@author: rj
"""
#%% IMPORTS
#import xlrd
import spacy
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import random
import gensim
from gensim import corpora
import pickle
import pyLDAvis.gensim

spacy.load('en')
from spacy.lang.en import English
parser = English()

nltk.download('stopwords')
nltk.download('wordnet')
en_stop = set(nltk.corpus.stopwords.words('english'))

#%% INPUT DATA

loc = ('/Users/rj/Documents/Hacking_4_Defense/python_code/Example IGEMS data.xlsx') #Insert filepath to the IGEMS Data

"""
wb = xlrd.open_workbook(loc)
'sheet = wb.sheet_by_index(0)
"""

data = pd.read_excel(loc)
df = pd.DataFrame(data, columns= ['Index','Name','Organization','Type','Comment'])
df = df.dropna() #Get rid of NaN rows
print(df)


#%% FUNCTIONS
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

#%% PROCESS DATA
#Completely ripped from:
#https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21    
text_data = []


UniqueTypes = df.Type.unique()
DataFrameDict = {elem : df for elem in UniqueTypes}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df[:][df.Type == key]

deficiency_df = pd.DataFrame.from_dict(DataFrameDict['Deficiency'])
recommendations_df = pd.DataFrame.from_dict(DataFrameDict['Recommended Improvement Area'])
grade_summary_df = pd.DataFrame.from_dict(DataFrameDict['Grade/Summary'])
strength_df = pd.DataFrame.from_dict(DataFrameDict['Strength'])


"""
# sort the dataframe
df.sort_values(by='Type', axis=1, inplace=True)

# set the index to be this and don't drop
df.set_index(keys=['Type'], drop=False,inplace=True)
# get a list of names
types=df['Type'].unique().tolist()
# now we can perform a lookup on a 'view' of the dataframe
deficieny = df.loc[df.type=='deficiency']
# now you can query all 'joes'
"""




for ind in recommendations_df.index:
    tokens = prepare_text_for_lda(recommendations_df['Comment'][ind])
    #if random.random() > .99: #Something is really messed up with this line, why random?
    print(tokens)
    text_data.append(tokens)

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]


pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')




"""
NUM_TOPICS = 8
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model8.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)
"""
  

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)



dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)



[nltk_data] Downloading package stopwords to /Users/rj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/rj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


         Index                                   Name Organization  \
1      1.4.2.2                          Right Quality      anon_S3   
2      1.2.3.3                                   Unit      anon_S8   
3      1.2.3.3                                   Unit      anon_S8   
4      1.3.2.1                     Key Work Processes      anon_S8   
5        1.4.3  Mission-Assurance Command and Control     anon_S14   
6      1.4.1.3                         Right Quantity     anon_S20   
7    1.3.3.1.1                Self-Assessment Program     anon_S23   
8      1.1.2.1                               Manpower     anon_S26   
10   1.3.3.1.1                Self-Assessment Program     anon_S36   
11       1.1.1                               Adequacy     anon_S40   
12     1.2.3.1                             Individual     anon_S45   
13   1.3.3.2.3                              Relevance     anon_S48   
14     1.3.1.2                     Strategic Planning     anon_S51   
15     1.4.3.1     W

/Users/rj/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


# Word Cloud

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 24 16:04:44 2019

@author: rj
"""
#%% IMPORTS
#import xlrd
import spacy
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import random
import gensim
from gensim import corpora
import pickle
import pyLDAvis.gensim

spacy.load('en')
from spacy.lang.en import English
parser = English()

nltk.download('stopwords')
nltk.download('wordnet')
en_stop = set(nltk.corpus.stopwords.words('english'))

#%% INPUT DATA

#loc = ('./Example IGEMS data.xlsx') #Insert filepath to the IGEMS Data
loc = ('/Users/rj/Documents/Hacking_4_Defense/python_code/Example IGEMS data.xlsx') #Insert filepath to the IGEMS Data

"""
wb = xlrd.open_workbook(loc)
'sheet = wb.sheet_by_index(0)
"""

data = pd.read_excel(loc)
df = pd.DataFrame(data, columns= ['Index','Name','Organization','Type','Comment'])
df = df.dropna() #Get rid of NaN rows
print(df)

         Index                                   Name Organization  \
1      1.4.2.2                          Right Quality      anon_S3   
2      1.2.3.3                                   Unit      anon_S8   
3      1.2.3.3                                   Unit      anon_S8   
4      1.3.2.1                     Key Work Processes      anon_S8   
5        1.4.3  Mission-Assurance Command and Control     anon_S14   
6      1.4.1.3                         Right Quantity     anon_S20   
7    1.3.3.1.1                Self-Assessment Program     anon_S23   
8      1.1.2.1                               Manpower     anon_S26   
10   1.3.3.1.1                Self-Assessment Program     anon_S36   
11       1.1.1                               Adequacy     anon_S40   
12     1.2.3.1                             Individual     anon_S45   
13   1.3.3.2.3                              Relevance     anon_S48   
14     1.3.1.2                     Strategic Planning     anon_S51   
15     1.4.3.1     W

[nltk_data] Downloading package stopwords to /Users/rj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/rj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
